### For Updating Usage Rates As Per Model

Cell energy and usage rates must be hard coded in the file **'usage.xlsx'**

#### NOTE: For the cars in the 'model' column ('M3-E3, MS-E1, MS, MX) can put 1 as cell energy, which has no effect on kwh calculations

In [175]:
from sqlalchemy import create_engine
from sqlalchemy import(Table, Column, String, Integer, Boolean)
from sqlalchemy import MetaData, Table

import datetime
import regex as re
import pandas as pd

# Connecting to Engine
engine = create_engine('sqlite:///metals_dashboard.sqlite')

connection = engine.connect()

# What's in the database already?
print(engine.table_names())

['EVSales', 'M3E1', 'M3E3', 'MS', 'MX', 'battery_cap', 'cell18650H', 'cell2170C', 'cellM48', 'commodities', 'fastmarkets', 'usage']


In [176]:
df = pd.read_sql_query('SELECT * from commodities;', connection)
df['month_year'] = pd.to_datetime(df['Date']).dt.to_period('M')

df

Date  LME Ni cash price  LME Ni inventory (total)  \
0     2015-01-02 00:00:00.000000           14756.00                  414900.0   
1     2015-01-05 00:00:00.000000           15126.00                  414756.0   
2     2015-01-06 00:00:00.000000           15191.00                  415338.0   
3     2015-01-07 00:00:00.000000           15483.00                  415812.0   
4     2015-01-08 00:00:00.000000           15486.00                  415842.0   
5     2015-01-09 00:00:00.000000           15235.50                  415164.0   
6     2015-01-12 00:00:00.000000           15029.00                  414732.0   
7     2015-01-13 00:00:00.000000           14585.00                  416436.0   
8     2015-01-14 00:00:00.000000           14241.00                  418320.0   
9     2015-01-15 00:00:00.000000           14413.50                  418332.0   
10    2015-01-16 00:00:00.000000           14722.00                  418518.0   
11    2015-01-19 00:00:00.000000           14417.00                  417900.0   
12    2015-01-20 00:00:00.000000           14721.00                  418062.0   
13    2015-01-21 00:00:00.000000           14971.50                  419964.0   
14    2015-01-22 00:00:00.000000           14792.00                  421842.0   
15    2015-01-23 00:00:00.000000           14293.00                  423036.0   
16    2015-01-26 00:00:00.000000           14692.00                  424344.0   
17    2015-01-27 00:00:00.000000           14730.50                  424110.0   
18    2015-01-28 00:00:00.000000           14993.00                  423726.0   
19    2015-01-29 00:00:00.000000           14846.00                  425562.0   
20    2015-01-30 00:00:00.000000           15109.00                  426240.0   
21    2015-02-02 00:00:00.000000           15274.00                  424974.0   
22    2015-02-03 00:00:00.000000           15255.50                  424572.0   
23    2015-02-04 00:00:00.000000           15070.00                  424788.0   
24    2015-02-05 00:00:00.000000           15122.75                  423480.0   
25    2015-02-06 00:00:00.000000           15179.00                  424206.0   
26    2015-02-09 00:00:00.000000           15098.75                  426324.0   
27    2015-02-10 00:00:00.000000           14764.75                  426090.0   
28    2015-02-11 00:00:00.000000           14690.50                  426018.0   
29    2015-02-12 00:00:00.000000           14648.00                  425310.0   
...                          ...                ...                       ...   
1166  2019-06-24 00:00:00.000000           12069.50                  168876.0   
1167  2019-06-25 00:00:00.000000           12234.00                  168252.0   
1168  2019-06-26 00:00:00.000000           12424.00                  164838.0   
1169  2019-06-27 00:00:00.000000           12658.00                  165480.0   
1170  2019-06-28 00:00:00.000000           12617.00                  164718.0   
1171  2019-07-01 00:00:00.000000           12266.00                  161658.0   
1172  2019-07-02 00:00:00.000000           12004.50                  161448.0   
1173  2019-07-03 00:00:00.000000           12271.50                  158292.0   
1174  2019-07-04 00:00:00.000000           12270.00                  157386.0   
1175  2019-07-05 00:00:00.000000           12418.00                  157464.0   
1176  2019-07-08 00:00:00.000000           12676.00                  154740.0   
1177  2019-07-09 00:00:00.000000           12646.50                  154104.0   
1178  2019-07-10 00:00:00.000000           12928.00                  153612.0   
1179  2019-07-11 00:00:00.000000           13073.00                  153420.0   
1180  2019-07-12 00:00:00.000000           13409.50                  150840.0   
1181  2019-07-15 00:00:00.000000           13595.00                  150324.0   
1182  2019-07-16 00:00:00.000000           14004.00                  149322.0   
1183  2019-07-17 00:00:00.000000           14419.00        

In [177]:
baselines = df.groupby(['month_year']).mean()
baselines.head()

LME Ni cash price  LME Ni inventory (total)  \
month_year                                                
2015-01          14849.190476             419187.428571   
2015-02          14573.837500             425617.800000   
2015-03          13755.500000             431802.272727   
2015-04          12844.204545             434358.818182   
2015-05          13480.380952             448753.142857   

            LME Ni inventory (briquette)  LME Ni inventory (other forms)  \
month_year                                                                 
2015-01                    167046.571429                   252140.857143   
2015-02                    164840.400000                   260777.400000   
2015-03                    164323.363636                   267478.909091   
2015-04                    162215.181818                   272143.636364   
2015-05                    170423.142857                   278330.000000   

            Critical Inventory  LME Co cash price  LME Cu cash price  \
month_year                                                             
2015-01               100000.0       30749.476190        5830.535714   
2015-02               100000.0       29155.375000        5729.275000   
2015-03               100000.0       27568.784091        5939.670455   
2015-04               100000.0       28657.977273        6038.556818   
2015-05               100000.0       30084.095238        6293.726190   

            Comex Cu future  Comex Cu future2  LME Al cash price  \
month_year                                                         
2015-01          253.278571       5578.823159        1814.719048   
2015-02          251.210000       5533.259912        1817.820000   
2015-03          260.011364       5727.122547        1773.863636   
2015-04          266.093182       5861.083300        1815.738636   
2015-05          279.104762       6147.681980        1805.178571   

            Fastmarkets- Cobalt LOW  Fastmarkets - Cobalt High  
month_year                                                      
2015-01                         NaN                        NaN  
2015-02                         NaN                        NaN  
2015-03                         NaN                        NaN  
2015-04                         NaN                        NaN  
2015-05                         NaN                        NaN

In [178]:
df = pd.read_sql_query('SELECT * from usage;', connection)
df

index       Model       LiOH     Nickel    Cobalt     Copper     Aluminum  \
0      0       M3-E3  57.073451  68.830836  4.630438  50.334320   421.749205   
1      1       M3-E1  38.462543  46.385998  3.120513  43.870520   418.569640   
2      2          MS  75.923629  83.256088  9.063918  20.850000  1050.520000   
3      3          MX  75.923629  83.256088  9.063918  28.320000  1152.800000   
4      4     cellM48   0.011220   0.012980  0.001720   0.005960     0.002380   
5      5   cell2170C   0.011833   0.014868  0.000699   0.004503     0.002215   
6      6  cell18650H   0.009196   0.010084  0.001098   0.002915     0.001314   

        Steel  Manganese  cell_energy  
0  719.053646    0.00000          1.0  
1  695.557892    0.00000          1.0  
2   74.500000    0.00000          1.0  
3   74.500000    0.00000          1.0  
4    0.000000    0.00085         17.4  
5    0.000000    0.00000         17.6  
6    0.000000    0.00000         12.8

In [179]:
baselines

LME Ni cash price  LME Ni inventory (total)  \
month_year                                                
2015-01          14849.190476             419187.428571   
2015-02          14573.837500             425617.800000   
2015-03          13755.500000             431802.272727   
2015-04          12844.204545             434358.818182   
2015-05          13480.380952             448753.142857   
2015-06          12825.227273             463235.454545   
2015-07          11413.097826             455404.695652   
2015-08          10369.190476             455786.285714   
2015-09           9937.545455             451917.818182   
2015-10          10316.829545             438440.727273   
2015-11           9244.333333             417454.571429   
2015-12           8698.369565             423081.391304   
2016-01           8520.273810             443329.142857   
2016-02           8298.500000             440177.142857   
2016-03           8708.119565             432356.086957   
2016-04           8878.857143             425107.142857   
2016-05           8681.590909             408805.636364   
2016-06           8928.352273             389538.272727   
2016-07          10262.857143             376704.285714   
2016-08          10310.826087             371629.565217   
2016-09          10191.784091             365939.727273   
2016-10          10259.738095             361644.857143   
2016-11          11128.909091             365342.181818   
2016-12          10916.045455             370125.000000   
2017-01           9971.125000             373921.636364   
2017-02          10643.300000             381745.200000   
2017-03          10204.663043             381883.304348   
2017-04           9617.875000             375180.900000   
2017-05           9160.934783             380613.391304   
2017-06           8931.761364             377251.909091   
2017-07           9491.392857             374063.142857   
2017-08          10913.282609             380951.739130   
2017-09          11215.785714             383604.857143   
2017-10          11335.772727             385330.636364   
2017-11          11972.000000             381571.636364   
2017-12          11550.095238             375168.285714   
2018-01          12857.945652             364072.173913   
2018-02          13595.875000             341649.600000   
2018-03          13386.159091             326761.090909   
2018-04          13905.476190             314293.714286   
2018-05          14364.967391             305134.173913   
2018-06          15105.654762             277338.285714   
2018-07          13793.863636             263033.727273   
2018-08          13408.163043             246933.913043   
2018-09          12510.350000             233426.400000   
2018-10          12314.913043             222923.478261   
2018-11          11239.715909             215937.818182   
2018-12          10831.833333             210002.857143   
2019-01          11483.152174             202911.391304   
2019-02          12685.225000             199932.000000   
2019-03          13026.273810             190370.285714   
2019-04          12755.988636             177783.545455   
2019-05          12035.913043             167191.826087   
2019-06          11943.937500             165138.000000   
2019-07          13546.304348             150811.826087   
2019-08          14578.000000             143670.000000   

            LME Ni inventory (briquette)  LME Ni inventory (other forms)  \
month_year                                                                 
2015-01                    167046.571429                   252140.857143   
2015-02                    164840.400000                   260777.400000   
2015-03                    164323.363636                   267478.909091   
2015-04                    162215.181818                   272143.636364   
2015-05                    170423.142857                   278330.000000   
2015-06                    178053.000000                 

### Cleaning

In [180]:
df

index       Model       LiOH     Nickel    Cobalt     Copper     Aluminum  \
0      0       M3-E3  57.073451  68.830836  4.630438  50.334320   421.749205   
1      1       M3-E1  38.462543  46.385998  3.120513  43.870520   418.569640   
2      2          MS  75.923629  83.256088  9.063918  20.850000  1050.520000   
3      3          MX  75.923629  83.256088  9.063918  28.320000  1152.800000   
4      4     cellM48   0.011220   0.012980  0.001720   0.005960     0.002380   
5      5   cell2170C   0.011833   0.014868  0.000699   0.004503     0.002215   
6      6  cell18650H   0.009196   0.010084  0.001098   0.002915     0.001314   

        Steel  Manganese  cell_energy  
0  719.053646    0.00000          1.0  
1  695.557892    0.00000          1.0  
2   74.500000    0.00000          1.0  
3   74.500000    0.00000          1.0  
4    0.000000    0.00085         17.4  
5    0.000000    0.00000         17.6  
6    0.000000    0.00000         12.8

In [181]:
for key, model in df['Model'].items():
    df.iloc[key, 1]= model.replace('-', '')
    print(model.replace('-', ''))
    vars()[str(model)] = baselines[['LME Ni cash price', 'LME Co cash price', 'LME Cu cash price', 'LME Al cash price']]
    
    for metal in df.columns[3:7]:
        #print('usage ', df.loc[key, metal])
        
        
        
        if metal != 'Copper':
            symbol = metal[:2]
            col_name = 'LME ' + symbol + ' cash price'
            new_col_name = symbol + ' usage'
            '''Populate columns in dataframe'''
            #print(str(model)[col_name]*df.loc[key, metal])
            vars()[str(model)][new_col_name] = (vars()[str(model)][col_name]*df.loc[key, metal])/(1000*df.loc[key, 'cell_energy'])
        else:
            symbol = 'Cu'
            col_name = 'LME ' + symbol + ' cash price'
            new_col_name = symbol + ' usage'
            '''Populate columns in dataframe'''
            #print(str(model)[col_name]*df.loc[key, metal])
            vars()[str(model)][new_col_name] = (vars()[str(model)][col_name]*df.loc[key, metal])/(1000*df.loc[key, 'cell_energy'])
        
        #nickel_usage = 
        #print(item)
        
    
    #print(item)

M3E3
M3E1


C:\Users\ckato\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\ckato\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


MS
MX
cellM48
cell2170C
cell18650H


In [182]:
M3E3.to_csv('raw_data_finals/usage_per_model/M3E3.csv')
M3E1.to_csv('raw_data_finals/usage_per_model/M3E1.csv')
MS.to_csv('raw_data_finals/usage_per_model/MS.csv')
MX.to_csv('raw_data_finals/usage_per_model/MX.csv')
cellM48.to_csv('raw_data_finals/usage_per_model/cellM48.csv')
cell2170C.to_csv('raw_data_finals/usage_per_model/cell2170C.csv')
cell18650H.to_csv('raw_data_finals/usage_per_model/cell18650H.csv')


In [185]:
M3E3df = pd.read_csv('raw_data_finals/usage_per_model/M3E3.csv')
M3E1df = pd.read_csv('raw_data_finals/usage_per_model/M3E1.csv')
MSdf = pd.read_csv('raw_data_finals/usage_per_model/MS.csv')
MXdf = pd.read_csv('raw_data_finals/usage_per_model/MX.csv')
cellM48df = pd.read_csv('raw_data_finals/usage_per_model/cellM48.csv')
cell2170Cdf = pd.read_csv('raw_data_finals/usage_per_model/cell2170C.csv')
cell18650df=pd.read_csv('raw_data_finals/usage_per_model/cell18650H.csv')

M3E3df['month_year'] = pd.to_datetime(M3E3df['month_year'])
M3E1df['month_year'] = pd.to_datetime(M3E1df['month_year'])
MSdf['month_year'] = pd.to_datetime(MSdf['month_year'])
MXdf['month_year'] = pd.to_datetime(MXdf['month_year'])
cellM48df['month_year'] = pd.to_datetime(cellM48df['month_year'])
cell2170Cdf['month_year'] = pd.to_datetime(cell2170Cdf['month_year'])
cell18650df['month_year'] = pd.to_datetime(cell18650df['month_year'])

M3E3df.index = M3E3df['month_year']
M3E1df.index = M3E3df['month_year']
MSdf.index = M3E3df['month_year']
MXdf.index = M3E3df['month_year']
cellM48df.index = M3E3df['month_year']
cell2170Cdf.index = M3E3df['month_year']
cell18650df.index = M3E3df['month_year']


M3E3df = M3E3df.iloc[:, 1:len(M3E3df.columns)]
M3E1df = M3E1df.iloc[:, 1:len(M3E1df.columns)]
MSdf = MSdf.iloc[:, 1:len(MSdf.columns)]
MXdf = MXdf.iloc[:, 1:len(MXdf.columns)]
cellM48df = cellM48df.iloc[:, 1:len(cellM48df.columns)]
cell2170Cdf = cell2170Cdf.iloc[:, 1:len(cell2170Cdf.columns)]
cell18650df = cell18650df.iloc[:, 1:len(cell18650df.columns)]


In [186]:
cell18650df

LME Ni cash price  LME Co cash price  LME Cu cash price  \
month_year                                                            
2015-01-01       14849.190476       30749.476190        5830.535714   
2015-02-01       14573.837500       29155.375000        5729.275000   
2015-03-01       13755.500000       27568.784091        5939.670455   
2015-04-01       12844.204545       28657.977273        6038.556818   
2015-05-01       13480.380952       30084.095238        6293.726190   
2015-06-01       12825.227273       30650.386364        5833.011364   
2015-07-01       11413.097826       31579.869565        5456.750000   
2015-08-01       10369.190476       29191.333333        5128.238095   
2015-09-01        9937.545455       27714.306818        5217.250000   
2015-10-01       10316.829545       27499.045455        5216.090909   
2015-11-01        9244.333333       24413.702381        4799.904762   
2015-12-01        8698.369565       24016.728261        4643.326087   
2016-01-01        8520.273810       23173.428571        4482.928571   
2016-02-01        8298.500000       22275.880952        4598.619048   
2016-03-01        8708.119565       22973.260870        4955.815217   
2016-04-01        8878.857143       23000.500000        4872.738095   
2016-05-01        8681.590909       23439.363636        4712.511364   
2016-06-01        8928.352273       23692.727273        4641.965909   
2016-07-01       10262.857143       25085.523810        4864.904762   
2016-08-01       10310.826087       26283.695652        4745.228261   
2016-09-01       10191.784091       26625.000000        4722.204545   
2016-10-01       10259.738095       28205.666667        4731.261905   
2016-11-01       11128.909091       29258.409091        5450.931818   
2016-12-01       10916.045455       31692.886364        5641.750000   
2017-01-01        9971.125000       34630.090909        5744.034091   
2017-02-01       10643.300000       43005.000000        5940.912500   
2017-03-01       10204.663043       52765.217391        5824.630435   
2017-04-01        9617.875000       55300.000000        5681.512500   
2017-05-01        9160.934783       54983.945652        5606.054348   
2017-06-01        8931.761364       57449.454545        5719.761364   
2017-07-01        9491.392857       58548.904762        5985.119048   
2017-08-01       10913.282609       58564.478261        6492.706522   
2017-09-01       11215.785714       60128.500000        6577.166667   
2017-10-01       11335.772727       59905.863636        6807.602273   
2017-11-01       11972.000000       62118.045455        6826.545455   
2017-12-01       11550.095238       72877.095238        6857.809524   
2018-01-01       12857.945652       77202.717391        7071.989130   
2018-02-01       13595.875000       80792.600000        7006.525000   
2018-03-01       13386.159091       87884.681818        6793.727273   
2018-04-01       13905.476190       90914.285714        6843.309524   
2018-05-01       14364.967391       90195.652174        6825.478261   
2018-06-01       15105.654762       81188.095238        6965.857143   
2018-07-01       13793.863636       70647.727273        6250.750000   
2018-08-01       13408.163043       63324.195652        6052.804348   
2018-09-01       12510.350000       62210.325000        6050.762500   
2018-10-01       12314.913043       60626.782609        6219.586957   
2018-11-01       11239.715909       54962.931818        6195.920455   
2018-12-01       10831.833333       55238.095238        6061.642857   
2019-01-01       11483.152174       41326.086957        5939.532609   
2019-02-01       12685.225000       32100.000000        6300.487500   
2019-03-01       13026.273810       31333.333333        6439.464286   
2019-04-01       12755.988636       33636.363636        6440.397727   
2019-05-01       12035.913043       34152.173913        6023.358696   
2019-06-01       11943.937500       28809.000000        5882.225000   
2019-07-01       13546.304348       

In [187]:
M3E3df.to_sql("M3E3", con=engine, if_exists='replace', index = True)
M3E1df.to_sql("M3E1", con=engine, if_exists='replace', index = True)
MSdf.to_sql("MS", con=engine, if_exists='replace', index = True)
MXdf.to_sql("MX", con=engine, if_exists='replace', index = True)
cellM48df.to_sql("cellM48", con=engine, if_exists='replace', index = True)
cell2170Cdf.to_sql("cell2170C", con=engine, if_exists='replace', index = True)
cell18650df.to_sql("cell18650H", con=engine, if_exists='replace', index = True)


In [159]:
cell2170Cdf

month_year  LME Ni cash price  LME Co cash price  LME Cu cash price  \
0  2015-01-01       14849.190476       30749.476190        5830.535714   
1  2015-02-01       14573.837500       29155.375000        5729.275000   
2  2015-03-01       13755.500000       27568.784091        5939.670455   
3  2015-04-01       12844.204545       28657.977273        6038.556818   
4  2015-05-01       13480.380952       30084.095238        6293.726190   
5  2015-06-01       12825.227273       30650.386364        5833.011364   
6  2015-07-01       11413.097826       31579.869565        5456.750000   
7  2015-08-01       10369.190476       29191.333333        5128.238095   
8  2015-09-01        9937.545455       27714.306818        5217.250000   
9  2015-10-01       10316.829545       27499.045455        5216.090909   
10 2015-11-01        9244.333333       24413.702381        4799.904762   
11 2015-12-01        8698.369565       24016.728261        4643.326087   
12 2016-01-01        8520.273810       23173.428571        4482.928571   
13 2016-02-01        8298.500000       22275.880952        4598.619048   
14 2016-03-01        8708.119565       22973.260870        4955.815217   
15 2016-04-01        8878.857143       23000.500000        4872.738095   
16 2016-05-01        8681.590909       23439.363636        4712.511364   
17 2016-06-01        8928.352273       23692.727273        4641.965909   
18 2016-07-01       10262.857143       25085.523810        4864.904762   
19 2016-08-01       10310.826087       26283.695652        4745.228261   
20 2016-09-01       10191.784091       26625.000000        4722.204545   
21 2016-10-01       10259.738095       28205.666667        4731.261905   
22 2016-11-01       11128.909091       29258.409091        5450.931818   
23 2016-12-01       10916.045455       31692.886364        5641.750000   
24 2017-01-01        9971.125000       34630.090909        5744.034091   
25 2017-02-01       10643.300000       43005.000000        5940.912500   
26 2017-03-01       10204.663043       52765.217391        5824.630435   
27 2017-04-01        9617.875000       55300.000000        5681.512500   
28 2017-05-01        9160.934783       54983.945652        5606.054348   
29 2017-06-01        8931.761364       57449.454545        5719.761364   
30 2017-07-01        9491.392857       58548.904762        5985.119048   
31 2017-08-01       10913.282609       58564.478261        6492.706522   
32 2017-09-01       11215.785714       60128.500000        6577.166667   
33 2017-10-01       11335.772727       59905.863636        6807.602273   
34 2017-11-01       11972.000000       62118.045455        6826.545455   
35 2017-12-01       11550.095238       72877.095238        6857.809524   
36 2018-01-01       12857.945652       77202.717391        7071.989130   
37 2018-02-01       13595.875000       80792.600000        7006.525000   
38 2018-03-01       13386.159091       87884.681818        6793.727273   
39 2018-04-01       13905.476190       90914.285714        6843.309524   
40 2018-05-01       14364.967391       90195.652174        6825.478261   
41 2018-06-01       15105.654762       81188.095238        6965.857143   
42 2018-07-01       13793.863636       70647.727273        6250.750000   
43 2018-08-01       13408.163043       63324.195652        6052.804348   
44 2018-09-01       12510.350000       62210.325000        6050.762500   
45 2018-10-01       12314.913043       60626.782609        6219.586957   
46 2018-11-01       11239.715909       54962.931818        6195.920455   
47 2018-12-01       10831.833333       55238.095238        6061.642857   
48 2019-01-01       11483.152174       41326.086957        5939.532609   
49 2019-02-01       12685.225000       32100.000000        6300.487500   
50 2019-03-01       13026.273810       31333.333333        6439.464286   
51 2019-04-01       12755.988636       33636.363636        6440.397727   
52 2019-05-01       12035.913043       34152.173913        6023.358696   
53 2019

In [188]:
print(engine.table_names())

['EVSales', 'M3E1', 'M3E3', 'MS', 'MX', 'battery_cap', 'cell18650H', 'cell2170C', 'cellM48', 'commodities', 'fastmarkets', 'usage']


## Note: Need data for Mangenese, Steel, and Lithium